In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import time
from pathlib import Path
from tqdm import tqdm

In [2]:
def invoc_per_func(ruta = Path.cwd()):
    return  [arch.name for arch in Path(ruta).glob('invocations_per_function_md.anon.d[01][0-9].csv')]

invoc_func = invoc_per_func() 


In [3]:
def rango(df):
    ran = []
    for (index, fila) in df.iterrows():
        cont = 0
        fin = 0 
        for m in fila.values:
            cont+=1
            if(m == 1):
                ini = cont
                cont = 0
                break
        for m in reversed(fila.values):
            cont+=1
            if(m == 1):
                fin = 1441 - cont 
                cont = 0
                break
        if((fin - ini) == 0): # evitamos rango 0 en registros unicos por dia 
            ran.append(1)
            
        else:
            ran.append(fin - ini)
    
    df['rango_min'] = ran
    del ran

# metodo para resultado de los 14 dias
en el metodo anterior se analizaba los 14 dias pero solo se guadaba la primera incidencia ahora se guarda los calculos de cada funcion para todos los 14 dias, si una funcion que ya ha aparecido antes no parece otro dia sus valores seran None

In [4]:
def clean_laps_df2(df):
    tipo = df["Trigger"].to_numpy()
    df.drop(['HashOwner', 'HashApp', 'Trigger'], axis = 1, inplace = True)
    df.set_index('HashFunction',inplace=True) # para hacer sencillo el reemplazo de # de invocaciones por minuto 
    df=df.where(df <= 1, 1)                 # si se invocó 12veces en un minuto se reeplaza a 1 para analizar periodicidad 
    invocaciones = pd.Series(df.iloc[:, 0:].sum(axis=1)) # llevamos conteo de # de aparicicones en el dia de una funcion
    rango(df)
    df["ivocationDay"] = invocaciones
    df["periodo"] = df.apply(lambda x: 1440 if (x["ivocationDay"]==1) else (x["rango_min"]/(x["ivocationDay"]-1)) , axis = 1) 
        df["periodo"] = df["periodo"].round(2)
    df["Trigger"] = tipo
    df['HashFunction'] = df.index
    df.reset_index(drop = True, inplace = True)
    df = df[['HashFunction','Trigger','periodo']]

    return df # def con HashFuntion como index

In [5]:
# sacar el peridodo de todas las funciones en Dataset
def periodo_all(): 
    # cargamos df de todos los registros en los 14 dias 
    invoc_per_func = pd.read_csv("invocations_per_function_md.anon.d01.csv")
    with tqdm(total=len(invoc_func), file=sys.stdout) as pbar:
        cont_arch = 0
        for arch in invoc_func:
            if(arch == "invocations_per_function_md.anon.d01.csv"):
                invoc_per_func = clean_laps_df2(invoc_per_func)
            else:
                aux = pd.read_csv(arch)
                aux = clean_laps_df2(aux)
                invoc_per_func = invoc_per_func.merge(aux, left_on=['HashFunction', 'Trigger'], right_on=['HashFunction', 'Trigger'], how='outer',suffixes=("_"+str(cont_arch), "_"+str(cont_arch+1)))
                del aux
            cont_arch+=1    
            pbar.set_description('archivos procesados: %d' %(cont_arch))
            pbar.update(1)
    invoc_per_func
    moda = invoc_per_func.iloc[:, 2:].mode(axis=1).iloc[:, 0]
    desv = invoc_per_func.iloc[:, 2:].std(axis=1)
    invoc_per_func["periodo"] = moda.round(2)
    invoc_per_func["Desv_Est"] = desv.round(2)
    invoc_per_func = invoc_per_func[["HashFunction","Trigger","periodo","Desv_Est"]]
    return (invoc_per_func)


In [6]:
aux = periodo_all()


archivos procesados: 14: 100%|█████████████████████████████████████████████████████████| 14/14 [05:07<00:00, 21.94s/it]


In [7]:
aux

,HashFunction,Trigger,periodo,Desv_Est
0,520dbd6bd906840012aa0c4b778743efc7c0ac7b7caf96...,http,5.0,0.00
1,115ca7a2b5bc290052c3da74cd0347d19c3c67b7d5aa66...,http,5.0,0.01
2,93e6c664773bbec3a7f50a0e92fa7e97401a802dc6eed8...,orchestration,1440.0,510.93
3,740c5c767e4b9978ee59a97d1829cfbaf755a47806a311...,http,1440.0,546.03
4,c108b4864b866b38b80d0e4594cc6d038f39668b804a1b...,http,1440.0,508.89
...,...,...,...,...
74342,4795a4eddf26cca87a5c7c1de760fc1beefd16f866c197...,http,1440.0,NaN
74343,75e947006f189b6cce5c6516975142f29a0f169e764fde...,storage,85.0,NaN
74344,11b3d8753f8f707b77c23063fb9cc9cfe9c74b40f9ab8c...,storage,1440.0,NaN
74345,42212390028151a3ab9918fd440633957a4b3b8bd3e679...,storage,1440.0,NaN


In [8]:
copia = aux.copy()
copia

,HashFunction,Trigger,periodo,Desv_Est
0,520dbd6bd906840012aa0c4b778743efc7c0ac7b7caf96...,http,5.0,0.00
1,115ca7a2b5bc290052c3da74cd0347d19c3c67b7d5aa66...,http,5.0,0.01
2,93e6c664773bbec3a7f50a0e92fa7e97401a802dc6eed8...,orchestration,1440.0,510.93
3,740c5c767e4b9978ee59a97d1829cfbaf755a47806a311...,http,1440.0,546.03
4,c108b4864b866b38b80d0e4594cc6d038f39668b804a1b...,http,1440.0,508.89
...,...,...,...,...
74342,4795a4eddf26cca87a5c7c1de760fc1beefd16f866c197...,http,1440.0,NaN
74343,75e947006f189b6cce5c6516975142f29a0f169e764fde...,storage,85.0,NaN
74344,11b3d8753f8f707b77c23063fb9cc9cfe9c74b40f9ab8c...,storage,1440.0,NaN
74345,42212390028151a3ab9918fd440633957a4b3b8bd3e679...,storage,1440.0,NaN


In [9]:
copia.dropna(inplace = True)
copia

,HashFunction,Trigger,periodo,Desv_Est
0,520dbd6bd906840012aa0c4b778743efc7c0ac7b7caf96...,http,5.00,0.00
1,115ca7a2b5bc290052c3da74cd0347d19c3c67b7d5aa66...,http,5.00,0.01
2,93e6c664773bbec3a7f50a0e92fa7e97401a802dc6eed8...,orchestration,1440.00,510.93
3,740c5c767e4b9978ee59a97d1829cfbaf755a47806a311...,http,1440.00,546.03
4,c108b4864b866b38b80d0e4594cc6d038f39668b804a1b...,http,1440.00,508.89
...,...,...,...,...
73914,cbd3427b2a6887a18e6d9191cb40812ecef3472f44234d...,timer,1.29,0.04
73915,9c8349b7cdce8cd04f20907361b98f40bc48f7cf9831c9...,http,3.67,0.87
73916,d6aa9971da3f1908cc0e23e5ad271d491e7f4bb455ee45...,http,3.64,0.89
73917,4f2b3d43ebd38a84fec3ca9919e01223ab53796634ca3c...,http,54.36,34.39


In [10]:
# antes de la ultima correccion 
aux.to_csv("Periodos_All.csv", index=False)